# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab
import numpy as np

A newer version of GraphLab Create (v1.8.1) is available! Your current version is v1.7.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('C:\Users\Jas\Documents\UofWash_ML\kc_house_data.gl/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [4]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [19]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 0.365410     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 0.396661     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [7]:
model_all.coefficients.sort('value',ascending = False)

name,index,value
(intercept),None,274873.05595
bathrooms,None,8468.53108691
grade,None,842.068034898
sqft_living_sqrt,None,350.060553386
sqft_living,None,24.4207209824
sqft_above,None,20.0247224171
floors_square,None,0.0
yr_renovated,None,0.0
yr_built,None,0.0
sqft_basement,None,0.0


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [23]:

for l1_penalty in np.logspace(1, 7, num=13):

    train_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False, solver ='fista',
                                              #max_iterations = 70,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    validate_predict = train_model.predict(validation)
    RSS_validation = np.dot(validation['price'] - validate_predict, 
                            validation['price'] - validate_predict)
    print 'l1_penalty = %e' % l1_penalty
    print 'number of nonzeros = %d' % (train_model.coefficients['value']).nnz()
    #print train_model.coefficients.sort('value',ascending = False)
    #print predict
    print 'RSS_validation of %e' % l1_penalty + ' is ' + str(RSS_validation)
    print '\n'

l1_penalty = 1.000000e+01
number of nonzeros = 18
RSS_validation of 1.000000e+01 is 6.25766285142e+14


l1_penalty = 3.162278e+01
number of nonzeros = 18
RSS_validation of 3.162278e+01 is 6.25766285362e+14


l1_penalty = 1.000000e+02
number of nonzeros = 18
RSS_validation of 1.000000e+02 is 6.25766286058e+14


l1_penalty = 3.162278e+02
number of nonzeros = 18
RSS_validation of 3.162278e+02 is 6.25766288257e+14


l1_penalty = 1.000000e+03
number of nonzeros = 18
RSS_validation of 1.000000e+03 is 6.25766295212e+14


l1_penalty = 3.162278e+03
number of nonzeros = 18
RSS_validation of 3.162278e+03 is 6.25766317206e+14


l1_penalty = 1.000000e+04
number of nonzeros = 18
RSS_validation of 1.000000e+04 is 6.25766386761e+14


l1_penalty = 3.162278e+04
number of nonzeros = 18
RSS_validation of 3.162278e+04 is 6.25766606749e+14


l1_penalty = 1.000000e+05
number of nonzeros = 18
RSS_validation of 1.000000e+05 is 6.25767302792e+14


l1_penalty = 3.162278e+05
number of nonzeros = 18
RSS_validation

In [26]:
 #test l1_penalty = 3570million on Lasso selecting 7 features:
l1_penalty = 3570000000
train_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False, solver ='fista',
                                              # max_iterations = 70,
                                              l2_penalty=0., l1_penalty=l1_penalty)
validate_predict = train_model.predict(validation)
RSS_validation = np.dot(validation['price'] - validate_predict, 
                            validation['price'] - validate_predict)
print 'l1_penalty = %e' % l1_penalty
print 'number of nonzeros = %d' % (train_model.coefficients['value']).nnz()
print train_model.coefficients.sort('value',ascending = False)
    #print predict
print 'RSS_validation of %e' % l1_penalty + ' is ' + str(RSS_validation)
print '\n'

l1_penalty = 3.570000e+09
number of nonzeros = 7
+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None | 228662.047885 |
|    bathrooms     |  None | 15191.7738817 |
|      grade       |  None | 2684.38219558 |
| sqft_living_sqrt |  None | 657.249579022 |
|     bedrooms     |  None | 180.981165671 |
|   sqft_living    |  None | 31.7994956898 |
|    sqft_above    |  None | 29.1543414242 |
|   yr_renovated   |  None |      0.0      |
|     yr_built     |  None |      0.0      |
|  sqft_basement   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
RSS_validation of 3.570000e+09 is 1.05991868843e+15




*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [33]:
for l1_penalty in np.logspace(9.53, 9.58, num=20):

    train_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False, solver ='fista',
                                              #max_iterations = 70,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    validate_predict = train_model.predict(validation)
    RSS_validation = np.dot(validation['price'] - validate_predict, 
                            validation['price'] - validate_predict)
    print 'l1_penalty = %e' % l1_penalty
    print 'number of nonzeros = %d' % (train_model.coefficients['value']).nnz()
    #print train_model.coefficients.sort('value',ascending = False)
    #print predict
    print 'RSS_validation of %e' % l1_penalty + ' is ' + str(RSS_validation)
    print '\n'

l1_penalty = 3.388442e+09
number of nonzeros = 8
RSS_validation of 3.388442e+09 is 1.04124690022e+15


l1_penalty = 3.409036e+09
number of nonzeros = 8
RSS_validation of 3.409036e+09 is 1.04358158588e+15


l1_penalty = 3.429755e+09
number of nonzeros = 7
RSS_validation of 3.429755e+09 is 1.0453760646e+15


l1_penalty = 3.450601e+09
number of nonzeros = 7
RSS_validation of 3.450601e+09 is 1.04707828852e+15


l1_penalty = 3.471573e+09
number of nonzeros = 7
RSS_validation of 3.471573e+09 is 1.04888828924e+15


l1_penalty = 3.492673e+09
number of nonzeros = 7
RSS_validation of 3.492673e+09 is 1.0512306825e+15


l1_penalty = 3.513901e+09
number of nonzeros = 7
RSS_validation of 3.513901e+09 is 1.05362054551e+15


l1_penalty = 3.535258e+09
number of nonzeros = 7
RSS_validation of 3.535258e+09 is 1.05603332313e+15


l1_penalty = 3.556744e+09
number of nonzeros = 7
RSS_validation of 3.556744e+09 is 1.0584364556e+15


l1_penalty = 3.578361e+09
number of nonzeros = 7
RSS_validation of 3.578361e

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [ ]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [ ]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [17]:
l1_penalty_min = 9.5
l1_penalty_max = 10
l1_penalty_values = np.logspace(l1_penalty_min,l1_penalty_max,num=20)

for l1_penalty in l1_penalty_values:
    train_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    validation_predict = train_model.predict(validation)
    RSS_validation_sum = sum((validation['price'] - validation_predict)**2)
    RSS_validation_dot = np.dot(validation['price'] - validation_predict, validation['price'] - validation_predict)
    print 'l1_penalty = %e' % l1_penalty
    print 'number of nonzeros = %d' % (train_model.coefficients['value']).nnz()
    #print train_model.coefficients.sort('value',ascending = False)
    #print predict
    print 'RSS_validation of %e' % l1_penalty + ' is ' + str(RSS_validation_sum) + 'dot' + str(RSS_validation_dot)
    print '\n'

l1_penalty = 3.162278e+09
number of nonzeros = 10
RSS_validation of 3.162278e+09 is 1.00194263832e+15dot1.00194263832e+15


l1_penalty = 3.359818e+09
number of nonzeros = 8
RSS_validation of 3.359818e+09 is 1.03825877376e+15dot1.03825877376e+15


l1_penalty = 3.569699e+09
number of nonzeros = 7
RSS_validation of 3.569699e+09 is 1.05988499033e+15dot1.05988499033e+15


l1_penalty = 3.792690e+09
number of nonzeros = 6
RSS_validation of 3.792690e+09 is 1.08186759232e+15dot1.08186759232e+15


l1_penalty = 4.029611e+09
number of nonzeros = 6
RSS_validation of 4.029611e+09 is 1.10869843807e+15dot1.10869843807e+15


l1_penalty = 4.281332e+09
number of nonzeros = 6
RSS_validation of 4.281332e+09 is 1.14511197515e+15dot1.14511197515e+15


l1_penalty = 4.548778e+09
number of nonzeros = 6
RSS_validation of 4.548778e+09 is 1.19210297917e+15dot1.19210297917e+15


l1_penalty = 4.832930e+09
number of nonzeros = 5
RSS_validation of 4.832930e+09 is 1.24492736032e+15dot1.24492736032e+15


l1_penalty = 5.

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [ ]:
l1_penalty_min = 
l1_penalty_max = 

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [ ]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?